In [1]:
import hvplot.pandas
import pandas as pd
import requests

In [2]:
from api_keys import geoapify_key

In [3]:
city_data_df = pd.read_csv(r"C:\Users\nicho\OneDrive\Desktop\Starter_Code (3)\Starter_Code\output_data\cities.csv")


In [4]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
2,2,west island,-12.1568,96.8225,24.99,73,40,8.23,CC,1690295667
3,3,yicheng,31.3705,119.8703,27.62,88,100,5.51,CN,1690295739
4,4,la passe,45.5549,-0.8967,24.13,56,11,7.94,FR,1690295739
...,...,...,...,...,...,...,...,...,...,...
595,595,igarka,67.4667,86.5833,20.44,67,6,3.89,RU,1690295663
596,596,gloucester,51.8657,-2.2431,20.76,53,75,0.89,GB,1690295913
597,597,puerto aysen,-45.4000,-72.7000,1.02,100,100,0.00,CL,1690295940
598,598,baykalovo,57.3981,63.7684,20.58,61,40,4.02,RU,1690295941


In [5]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
2,2,west island,-12.1568,96.8225,24.99,73,40,8.23,CC,1690295667
3,3,yicheng,31.3705,119.8703,27.62,88,100,5.51,CN,1690295739
4,4,la passe,45.5549,-0.8967,24.13,56,11,7.94,FR,1690295739


In [36]:
map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Humidity",
    geo = True,
    grid = True,
    title = "City Latitude vs. Humidity (2022-10-18)",
    xlabel = "Latitude",
    ylabel = "Humidity(%)",
    frame_width = 600,
    frame_height = 500,
    line_width = 5

)

title = "City Latitude vs. Humidity (2022-10-18)"
xlabel = "Latitude"
ylabel = "Humidity (%)"

map_plot_1

:Points   [Lat,Humidity]

In [7]:
city_drop = []

for i in range(len(city_data_df)):
    if city_data_df["Cloudiness"][i] > 0:
        city_drop.append(i)
    
vacation_spot = city_data_df.drop(city_drop)
vacation_spot.dropna(inplace = True)



In [8]:
vacation_spot.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,28.53,66,0,1.79,US,1690295495
1,1,hami,42.8000,93.4500,32.64,10,0,6.69,CN,1690295738
26,26,port lincoln,-34.7333,135.8667,11.87,81,0,3.68,AU,1690295744
27,27,olhao,37.0286,-7.8411,29.49,62,0,4.63,PT,1690295678
38,38,bredasdorp,-34.5322,20.0403,12.87,68,0,2.70,ZA,1690295685


In [12]:
hotel_df = vacation_spot.drop(columns = ["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"]).copy()

In [13]:
hotel_df.insert(5, "Hotel Name", 0)

hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
0,bethel,41.3712,-73.4140,66,US,0
1,hami,42.8000,93.4500,10,CN,0
26,port lincoln,-34.7333,135.8667,81,AU,0
27,olhao,37.0286,-7.8411,62,PT,0
38,bredasdorp,-34.5322,20.0403,68,ZA,0
...,...,...,...,...,...,...
564,coracao de jesus,-16.6853,-44.3650,26,BR,0
575,plettenberg bay,-34.0527,23.3716,59,ZA,0
577,naxos,37.1056,25.3764,50,GR,0
586,couzeix,45.8705,1.2383,52,FR,0


In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": "1",
    "apiKey": f"{geoapify_key}"
}

In [18]:
print("Starting hotel search")

Starting hotel search


In [22]:
for index, row in hotel_df.iterrows():
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    base_url = "https://api.geoapify.com/v2/places"
        
    name_address = requests.get(base_url, params=params).json()
    
    try:  
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
hotel_df
        

bethel - nearest hotel: Hampton Inn Danbury
hami - nearest hotel: Хами
port lincoln - nearest hotel: Boston Hotel
olhao - nearest hotel: Hotel Cidade de Olhão
bredasdorp - nearest hotel: Victoria Hotel
atafu village - nearest hotel: No hotel found
manilva - nearest hotel: Hostal Ocaña
east london - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
sao vicente - nearest hotel: Pousada Vitória
port elizabeth - nearest hotel: Waterford Hotel
leninskoye - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
darwin - nearest hotel: Mantra Pantanas Darwin
al bawiti - nearest hotel: Old Oasis Hotel
hamilton - nearest hotel: North Vista Manor
hermanus - nearest hotel: Aloe guest house
luena - nearest hotel: No hotel found
anderson - nearest hotel: La Quinta
san patricio - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
tindo

,City,Lat,Lng,Humidity,Country,Hotel Name
0,bethel,41.3712,-73.4140,66,US,Hampton Inn Danbury
1,hami,42.8000,93.4500,10,CN,Хами
26,port lincoln,-34.7333,135.8667,81,AU,Boston Hotel
27,olhao,37.0286,-7.8411,62,PT,Hotel Cidade de Olhão
38,bredasdorp,-34.5322,20.0403,68,ZA,Victoria Hotel
...,...,...,...,...,...,...
564,coracao de jesus,-16.6853,-44.3650,26,BR,No hotel found
575,plettenberg bay,-34.0527,23.3716,59,ZA,Milkwood Manor
577,naxos,37.1056,25.3764,50,GR,Chateau Zevgoli
586,couzeix,45.8705,1.2383,52,FR,Hôtel Campanile


In [39]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "ESRI",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.99,
    color = "City",
    hover_cols = "all"
)

map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)